In [4]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    

In [7]:
dir_train = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
dir_desc = '/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt'
dir_test = '/kaggle/input/house-prices-advanced-regression-techniques/test.csv'

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import scale
from sklearn.impute import SimpleImputer
from sklearn import metrics
from numpy import linspace
import numpy as np
df_train = pd.read_csv(dir_train, index_col='Id')
df_test = pd.read_csv(dir_test, index_col='Id')

In [8]:
df_test.info()

In [9]:
col_list = df_train.isnull().sum().sort_values(ascending=False) > 200
col_name = col_list[col_list ==True].index
col_name

In [10]:
df_train.drop(columns=col_name, inplace=True)
df_test.drop(columns=col_name, inplace=True)
df_train.info()

In [11]:
df_test.info()

In [13]:
y = df_train.iloc[:,-1]
X = df_train.iloc[:, :-1]

In [14]:
col_list_test= df_test.columns.tolist()
index_test = df_test.index
index_test

In [15]:
imp = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
X_arr = imp.fit_transform(df_test)
df_test = pd.DataFrame(X_arr,columns=col_list_test, index=index_test)
df_test.rename_axis('Id', inplace=True)
df_test = df_test.convert_dtypes(convert_string=False)
df_test.info()

In [16]:
col_list_X= X.columns.tolist()
imp = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
X_arr = imp.fit_transform(X)
X = pd.DataFrame(X_arr,columns=col_list_X)
X = X.convert_dtypes(convert_string=False)
X.info()

In [17]:
X_test = pd.get_dummies(df_test)
X_test.info()

In [18]:
X_train = pd.get_dummies(X)
X_train.info()

In [19]:
col = X_test.columns.tolist()
X_train = X_train[col]
X_train.info()

In [20]:
metrics.SCORERS.keys()

In [21]:
X_train_full, X_valid, y_train_full, y_valid = train_test_split(X_train, y, train_size=0.8, test_size=0.2, random_state=6)

In [22]:
kf = KFold(shuffle=True, random_state=42)
result_1 = {}
for k in range(1,25):
    neigh = KNeighborsRegressor(n_neighbors=k, weights='distance',p=1)
    score = cross_val_score(neigh, X_train_full, y_train_full, cv=kf, scoring='neg_root_mean_squared_error')
    result_1[k] = score.mean()
print(max(result_1, key=result_1.get))

In [23]:
result_1

In [24]:
X_train_sc = scale(X_train_full)
result_2 = {}
for k in range(1,25):
    neigh = KNeighborsRegressor(n_neighbors=k, weights='distance', p=1)
    score = cross_val_score(neigh, X_train_sc, y_train_full, cv=kf, scoring='neg_root_mean_squared_error')
    result_2[k] = score.mean()
print(max(result_2, key=result_2.get))

In [25]:
result_2 

In [26]:
X_valid_sc = scale(X_valid)

In [27]:
pred = KNeighborsRegressor(n_neighbors=5, weights='distance', p=1)
pred.fit(X_train_sc, y_train_full)
predictions_train = pred.predict(X_train_sc)
predictions = pred.predict(X_valid_sc)
predictions
metrics.mean_squared_error(y_train_full, predictions_train)

In [28]:
metrics.mean_squared_log_error(y_valid, predictions)

In [29]:
X_test_sc = scale(X_test)

In [30]:
result = pred.predict(X_test_sc)
result

In [31]:
result_full = np.round(result, 3)
result_full

In [33]:
out = pd.DataFrame({'Id': X_test.index, 'SalePrice':result_full})
out.to_csv('my_submissionsHousePricePrediction.csv', index=False)